In [1]:
!pip install segmentation_models_pytorch
!pip install -q -U albumentations
!pip install timm
!pip install fastai --upgrade

     |████████████████████████████████| 71kB 6.6MB/s 
     |████████████████████████████████| 245kB 19.6MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp37-none-any.whl size=12420 sha256=17ea27ab60b15a7b3f9f85941f1126d31fc781c5ba957de31f977b0effd413bf
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp37-none-any.whl size=60963 sha256=353561898ec2d1dd6218fa571ce91d1194a248e3d3ed70c054119d5c0bf2c9c6
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built efficientnet-pytorch pretrainedmodels
     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 37.6MB 115kB/s 
     |████████████████████████████████| 952kB 66.2MB/s 
     |████████████████████████████████| 204kB 13.

In [1]:
import segmentation_models_pytorch as smp
from tqdm import tqdm
import gc

import math
from torch.optim.optimizer import Optimizer, required

from fastai.vision.all import *

from sklearn.model_selection import GroupKFold, KFold
import torch
from torch import nn
import torchvision
import cv2
import os
import numpy as np
import pandas as pd

from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts, _LRScheduler
from scipy.ndimage.interpolation import zoom
import albumentations as A
from torch.nn import functional as F
from albumentations.pytorch import ToTensorV2

from pycocotools.coco import COCO

import matplotlib.pyplot as plt
import sys
import time
import random
import timm

import zipfile

In [2]:
CFG = {
    "mean": (0.485, 0.456, 0.406),
    "std": (0.229, 0.224, 0.225)
}

In [3]:
device = "cuda"

test_path = '/content/drive/MyDrive/boostcamp_imgseg/test.json'


MODEL_PATHS_unext = ['/content/drive/MyDrive/boostcamp_imgseg/models/unext/resnext50_0.pth',
                     '/content/drive/MyDrive/boostcamp_imgseg/models/unext/resnext50_1.pth',
                     '/content/drive/MyDrive/boostcamp_imgseg/models/unext/resnext50_2.pth',
                     '/content/drive/MyDrive/boostcamp_imgseg/models/unext/resnext50_3.pth',
                     '/content/drive/MyDrive/boostcamp_imgseg/models/unext/resnext50_4.pth']

MODEL_PATHS_resnext50 = ['/content/drive/MyDrive/boostcamp_imgseg/models/resnext50_32x4d/0_checkpoint.pt',
                            '/content/drive/MyDrive/boostcamp_imgseg/models/resnext50_32x4d/1_checkpoint.pt',
                            '/content/drive/MyDrive/boostcamp_imgseg/models/resnext50_32x4d/2_checkpoint.pt',
                            '/content/drive/MyDrive/boostcamp_imgseg/models/resnext50_32x4d/3_checkpoint.pt',
                            '/content/drive/MyDrive/boostcamp_imgseg/models/resnext50_32x4d/4_checkpoint.pt']    

MODEL_PATHS_seresnet101 = ['/content/drive/MyDrive/boostcamp_imgseg/models/se_resnet101/se_resnet101_0fold.pth',
                            '/content/drive/MyDrive/boostcamp_imgseg/models/se_resnet101/se_resnet101_1fold.pth',
                            '/content/drive/MyDrive/boostcamp_imgseg/models/se_resnet101/se_resnet101_2fold.pth',
                            '/content/drive/MyDrive/boostcamp_imgseg/models/se_resnet101/se_resnet101_3fold.pth',
                            '/content/drive/MyDrive/boostcamp_imgseg/models/se_resnet101/se_resnet101_4fold.pth',]                                  

SUBMISSION_PATH = "/content/drive/MyDrive/boostcamp_imgseg/submission.csv"
OUT_MASKS = f'/content/drive/MyDrive/boostcamp_imgseg/pseudo_masks_jy.zip'




In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
class FPN(nn.Module):
    def __init__(self, input_channels:list, output_channels:list):
        super().__init__()
        self.convs = nn.ModuleList(
            [nn.Sequential(nn.Conv2d(in_ch, out_ch*2, kernel_size=3, padding=1),
             nn.ReLU(inplace=True), nn.BatchNorm2d(out_ch*2),
             nn.Conv2d(out_ch*2, out_ch, kernel_size=3, padding=1))
            for in_ch, out_ch in zip(input_channels, output_channels)])
        
    def forward(self, xs:list, last_layer):
        hcs = [F.interpolate(c(x),scale_factor=2**(len(self.convs)-i),mode='bilinear') 
               for i,(c,x) in enumerate(zip(self.convs, xs))]
        hcs.append(last_layer)
        return torch.cat(hcs, dim=1)

class UnetBlock(Module):
    def __init__(self, up_in_c:int, x_in_c:int, nf:int=None, blur:bool=False,
                 self_attention:bool=False, **kwargs):
        super().__init__()
        self.shuf = PixelShuffle_ICNR(up_in_c, up_in_c//2, blur=blur, **kwargs)
        self.bn = nn.BatchNorm2d(x_in_c)
        ni = up_in_c//2 + x_in_c
        nf = nf if nf is not None else max(up_in_c//2,32)
        self.conv1 = ConvLayer(ni, nf, norm_type=None, **kwargs)
        self.conv2 = ConvLayer(nf, nf, norm_type=None,
            xtra=SelfAttention(nf) if self_attention else None, **kwargs)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, up_in:Tensor, left_in:Tensor) -> Tensor:
        s = left_in
        up_out = self.shuf(up_in)
        cat_x = self.relu(torch.cat([up_out, self.bn(s)], dim=1))
        return self.conv2(self.conv1(cat_x))
        
class _ASPPModule(nn.Module):
    def __init__(self, inplanes, planes, kernel_size, padding, dilation, groups=1):
        super().__init__()
        self.atrous_conv = nn.Conv2d(inplanes, planes, kernel_size=kernel_size,
                stride=1, padding=padding, dilation=dilation, bias=False, groups=groups)
        self.bn = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU()

        self._init_weight()

    def forward(self, x):
        x = self.atrous_conv(x)
        x = self.bn(x)

        return self.relu(x)

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

class ASPP(nn.Module):
    def __init__(self, inplanes=512, mid_c=256, dilations=[6, 12, 18, 24], out_c=None):
        super().__init__()
        self.aspps = [_ASPPModule(inplanes, mid_c, 1, padding=0, dilation=1)] + \
            [_ASPPModule(inplanes, mid_c, 3, padding=d, dilation=d,groups=4) for d in dilations]
        self.aspps = nn.ModuleList(self.aspps)
        self.global_pool = nn.Sequential(nn.AdaptiveMaxPool2d((1, 1)),
                        nn.Conv2d(inplanes, mid_c, 1, stride=1, bias=False),
                        nn.BatchNorm2d(mid_c), nn.ReLU())
        out_c = out_c if out_c is not None else mid_c
        self.out_conv = nn.Sequential(nn.Conv2d(mid_c*(2+len(dilations)), out_c, 1, bias=False),
                                    nn.BatchNorm2d(out_c), nn.ReLU(inplace=True))
        self.conv1 = nn.Conv2d(mid_c*(2+len(dilations)), out_c, 1, bias=False)
        self._init_weight()

    def forward(self, x):
        x0 = self.global_pool(x)
        xs = [aspp(x) for aspp in self.aspps]
        x0 = F.interpolate(x0, size=xs[0].size()[2:], mode='bilinear', align_corners=True)
        x = torch.cat([x0] + xs, dim=1)
        return self.out_conv(x)
    
    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

class UneXt50(nn.Module):
    def __init__(self, stride=1, **kwargs):
        super().__init__()
        #encoder
        # m = torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models',
        #                    'resnext50_32x4d_ssl')
        m = timm.create_model("swsl_resnext50_32x4d", pretrained=True)
        self.enc0 = nn.Sequential(m.conv1, m.bn1, nn.ReLU(inplace=True))
        self.enc1 = nn.Sequential(nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1),
                            m.layer1) #256
        self.enc2 = m.layer2 #512
        self.enc3 = m.layer3 #1024
        self.enc4 = m.layer4 #2048
        #aspp with customized dilatations
        self.aspp = ASPP(2048,256,out_c=512,dilations=[stride*1,stride*2,stride*3,stride*4])
        self.drop_aspp = nn.Dropout2d(0.5)
        #decoder
        self.dec4 = UnetBlock(512,1024,256)
        self.dec3 = UnetBlock(256,512,128)
        self.dec2 = UnetBlock(128,256,64)
        self.dec1 = UnetBlock(64,64,32)
        self.fpn = FPN([512,256,128,64],[16]*4)
        self.drop = nn.Dropout2d(0.1)
        self.final_conv = ConvLayer(32+16*4, 12, ks=1, norm_type=None, act_cls=None)
        
    def forward(self, x):
        enc0 = self.enc0(x)
        enc1 = self.enc1(enc0)
        enc2 = self.enc2(enc1)
        enc3 = self.enc3(enc2)
        enc4 = self.enc4(enc3)
        enc5 = self.aspp(enc4)
        dec3 = self.dec4(self.drop_aspp(enc5),enc3)
        dec2 = self.dec3(dec3,enc2)
        dec1 = self.dec2(dec2,enc1)
        dec0 = self.dec1(dec1,enc0)
        x = self.fpn([enc5, dec3, dec2, dec1], dec0)
        x = self.final_conv(self.drop(x))
        x = F.interpolate(x,scale_factor=2,mode='bilinear')
        return x

In [6]:
class CustomDataLoader(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'train', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join('/content/drive/MyDrive/boostcamp_imgseg', image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
        
        if (self.mode in ('train', 'val')):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)

            # Load the categories in a variable
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id + 1" 할당
            # Background = 0
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            # Unknown = 1, General trash = 2, ... , Cigarette = 11
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = category_names.index(className)
                masks = np.maximum(self.coco.annToMask(anns[i])*pixel_value, masks)
            masks = masks

            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            
            return images, masks, image_infos
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            
            return images, image_infos
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

In [7]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [8]:
def get_validation_augmentations():
    return A.Compose([
        A.Normalize(mean=CFG['mean'], std=CFG['std'], max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ],p=1.0)

In [9]:
test_ds = CustomDataLoader(test_path, mode="test", transform=get_validation_augmentations())

test_loader = DataLoader(test_ds, 
                          batch_size=1, 
                          num_workers=4,
                         shuffle=False,
                         collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [18]:
def inference(models, imgs, device):
    outs = None
    flip_outs = None
    flips = [[-1],[-2],[-2,-1]]
    # flips = [[-1]]
    for model in models:
        model.eval()
        if outs == None:
            outs = F.softmax(model(imgs.to(device).float()), dim=1).detach()
        else:
            outs += F.softmax(model(imgs.to(device).float()), dim=1).detach()
        
    # outs /= len(models)
        
    for flip in flips:
        flip_img = torch.flip(imgs, flip)
        tmp_outs = None
        for model in models:
            flip_out = F.softmax(model(flip_img.to(device).float()), dim=1).detach()
            flip_out = torch.flip(flip_out, flip)
            if tmp_outs == None:
                tmp_outs = flip_out
            else:
                tmp_outs += flip_out
        tmp_outs /= len(models)
        if flip_outs == None:
            flip_outs = tmp_outs
        else:
            flip_outs += tmp_outs
    # flip_outs /= len(flips)
    
    outs += flip_outs
    
    return outs/(len(models) + len(flips))

In [11]:
def test(models1, models2, models3, data_loader, device):
  
    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction.')
    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)
    pbar = tqdm(enumerate(data_loader), total=len(data_loader), position=0, leave=True)
    with torch.no_grad():
        with zipfile.ZipFile(OUT_MASKS, 'w') as mask_out:
            for step, (imgs, image_infos) in pbar:
                # print(imgs)
                # print(imgs)
                imgs = torch.stack(imgs)
                
                outs1 = inference(models1, imgs, device)
                outs2 = inference(models2, imgs, device)
                outs3 = inference(models3, imgs, device)

                outs = (outs1 * 0.4) + (outs2 * 0.3) + (outs3 * 0.3)

                oms = torch.argmax(outs, dim=1).detach().cpu().numpy()
 
                file_name = image_infos[0]['file_name'].split("/")
                file_name[0] += "_masks"
                file_name[1] = file_name[1][:-4]
                file_name = "/".join(file_name)

                m = cv2.imencode(".png", oms.squeeze())[1]
                mask_out.writestr(f"{file_name}.png", m)

                # resize (256 x 256)
                temp_mask = []
                for img, mask in zip(np.stack(imgs), oms):
                    # print(mask.shape)
                    transformed = transform(image=img, mask=mask)
                    mask = transformed['mask']
                    temp_mask.append(mask)

                oms = np.array(temp_mask)
                oms = oms.reshape([oms.shape[0], size*size]).astype(int)

                preds_array = np.vstack((preds_array, oms))

                file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    return file_names, preds_array

In [12]:
models_resnext50 = []
models_unext = []
models_seresnet101 = []
# models_resnext101 = []

## ResNext50

In [13]:
for path in MODEL_PATHS_resnext50:
    model = smp.DeepLabV3Plus("resnext50_32x4d", encoder_weights=None, in_channels=3, classes=12).to(device)
    checkpoint = torch.load(path)
    # model.load_state_dict(checkpoint['model'])
    model.load_state_dict(checkpoint)
    model.eval()
    model.to(device)
    models_resnext50.append(model)

## UneXt

In [14]:
for path in MODEL_PATHS_unext:
    model = UneXt50().to(device)
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model'])
    # model.load_state_dict(checkpoint)
    model.eval()
    model.to(device)
    models_unext.append(model)

## ResNet101

In [15]:
for path in MODEL_PATHS_seresnet101:
    model = smp.DeepLabV3Plus("se_resnet101", encoder_weights=None, in_channels=3, classes=12).to(device)
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model'])
    # model.load_state_dict(checkpoint)
    model.eval()
    model.to(device)
    models_seresnet101.append(model)

In [19]:
# test set에 대한 prediction
file_names, preds = test(models_resnext50, models_unext, models_seresnet101, test_loader, device)

Start prediction.


  0%|          | 0/837 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100%|██████████| 837/837 [23:21<00:00,  1.67s/it]

End prediction.


In [20]:
# PredictionString 대입

submission = pd.read_csv('/content/drive/MyDrive/boostcamp_imgseg/sample_submission.csv', index_col=None)

for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(SUBMISSION_PATH, index=False)